# MyBatis
- Code: https://github.com/mybatis/mybatis-3
- Configuration: https://mybatis.org/mybatis-3/configuration.html
- [MyBatis Generator](https://mybatis.org/generator/index.html)


actions:
- [mybatis.uxf](../../UML/JavaEE/mybatis.uxf)
- workbench\Java\JavaEE\example-springcloud\persistent\mybatis

More:
- [MyBatis.md](./MyBatis.md)

# mybatis-spring-boot-starter
- http://mybatis.org/spring-boot-starter/mybatis-spring-boot-autoconfigure

# SqlSessionFactoryBuilder


```
org.apache.ibatis.session.SqlSessionFactoryBuilder#build(java.io.InputStream, java.lang.String, java.util.Properties)
\-- org.apache.ibatis.builder.xml.XMLConfigBuilder#parse - RETURN Configuration
\--\-- org.apache.ibatis.builder.xml.XMLConfigBuilder#parseConfiguration
\-- org.apache.ibatis.session.SqlSessionFactoryBuilder#build(org.apache.ibatis.session.Configuration)
\--\-- org.apache.ibatis.session.defaults.DefaultSqlSessionFactory
```

```java title="org.apache.ibatis.builder.xml.XMLConfigBuilder#parseConfiguration"
  private void parseConfiguration(XNode root) {
try {
  // issue #117 read properties first
  propertiesElement(root.evalNode("properties"));                          // properties
  Properties settings = settingsAsProperties(root.evalNode("settings"));   // settings
  loadCustomVfs(settings);
  loadCustomLogImpl(settings);                                               // log
  typeAliasesElement(root.evalNode("typeAliases"));                        // type alias
  pluginElement(root.evalNode("plugins"));                                 // plugin
  objectFactoryElement(root.evalNode("objectFactory"));
  objectWrapperFactoryElement(root.evalNode("objectWrapperFactory"));
  reflectorFactoryElement(root.evalNode("reflectorFactory"));
  settingsElement(settings);
  // read it after objectFactory and objectWrapperFactory issue #631
  environmentsElement(root.evalNode("environments"));                     // environments
  databaseIdProviderElement(root.evalNode("databaseIdProvider"));
  typeHandlerElement(root.evalNode("typeHandlers"));                      // type handlers
  mapperElement(root.evalNode("mappers"));                                // mappers
} catch (Exception e) {
  throw new BuilderException("Error parsing SQL Mapper Configuration. Cause: " + e, e);
}
  }
```

```java title="org.apache.ibatis.builder.xml.XMLConfigBuilder#pluginElement"
org.apache.ibatis.session.Configuration#addInterceptor
```

```java title="org.apache.ibatis.builder.xml.XMLConfigBuilder#mapperElement"
CASE < package name="">
name: org.apache.ibatis.session.Configuration#addMappers(java.lang.String)

CASE <mapper xxx="">
  CASE resource: org.apache.ibatis.builder.xml.XMLMapperBuilder#parse
  CASE url: org.apache.ibatis.builder.xml.XMLMapperBuilder#parse
  CASE class: org.apache.ibatis.session.Configuration#addMapper
```


# SqlSession


```java title="org.apache.ibatis.session.SqlSessionFactory#openSession()"
\-- org.apache.ibatis.session.defaults.DefaultSqlSessionFactory#openSessionFromDataSource

  private SqlSession openSessionFromDataSource(ExecutorType execType, TransactionIsolationLevel level,
  boolean autoCommit) {
Transaction tx = null;
try {
  final Environment environment = configuration.getEnvironment();
  final TransactionFactory transactionFactory = getTransactionFactoryFromEnvironment(environment);
  tx = transactionFactory.newTransaction(environment.getDataSource(), level, autoCommit);            // transaction
  final Executor executor = configuration.newExecutor(tx, execType);                                 // executor
  return new DefaultSqlSession(configuration, executor, autoCommit);                                 // SqlSession
} catch (Exception e) {
  closeTransaction(tx); // may have fetched a connection so lets call close()
  throw ExceptionFactory.wrapException("Error opening session.  Cause: " + e, e);
} finally {
  ErrorContext.instance().reset();
}
  }
```

```java title="org.apache.ibatis.session.Configuration#newExecutor(org.apache.ibatis.transaction.Transaction, org.apache.ibatis.session.ExecutorType)"
  public Executor newExecutor(Transaction transaction, ExecutorType executorType) {
executorType = executorType == null ? defaultExecutorType : executorType;
Executor executor;
if (ExecutorType.BATCH == executorType) {
  executor = new BatchExecutor(this, transaction);
} else if (ExecutorType.REUSE == executorType) {
  executor = new ReuseExecutor(this, transaction);
} else {
  executor = new SimpleExecutor(this, transaction);
}
if (cacheEnabled) { // settings: cacheEnabled
  executor = new CachingExecutor(executor);
}
return (Executor) interceptorChain.pluginAll(executor); // plugin, exexutor
  }
```


# Mapper

## mapper proxy


```java title="org.apache.ibatis.session.SqlSession#getMapper"
\-- org.apache.ibatis.session.Configuration#getMapper
\--\-- org.apache.ibatis.binding.MapperRegistry#getMapper

  public <T> T getMapper(Class<T> type, SqlSession sqlSession) {
final MapperProxyFactory<T> mapperProxyFactory = (MapperProxyFactory<T>) knownMappers.get(type);
if (mapperProxyFactory == null) {
  throw new BindingException("Type " + type + " is not known to the MapperRegistry.");
}
try {
  return mapperProxyFactory.newInstance(sqlSession); // get proxy instance
} catch (Exception e) {
  throw new BindingException("Error getting mapper instance. Cause: " + e, e);
}
  }
```


## mapper methods


```java title="org.apache.ibatis.binding.MapperProxy#invoke"
\-- org.apache.ibatis.binding.MapperProxy#cachedInvoker
\-- org.apache.ibatis.binding.MapperProxy.MapperMethodInvoker#invoke
\--\-- org.apache.ibatis.binding.MapperMethod#execute


  public Object execute(SqlSession sqlSession, Object[] args) {
Object result;
switch (command.getType()) {
  case INSERT: {
    Object param = method.convertArgsToSqlCommandParam(args);
    result = rowCountResult(sqlSession.insert(command.getName(), param));
    break;
  }
  case UPDATE: {
    Object param = method.convertArgsToSqlCommandParam(args);
    result = rowCountResult(sqlSession.update(command.getName(), param));
    break;
  }
  case DELETE: {
    Object param = method.convertArgsToSqlCommandParam(args);
    result = rowCountResult(sqlSession.delete(command.getName(), param));
    break;
  }
  case SELECT:
    if (method.returnsVoid() && method.hasResultHandler()) {
      executeWithResultHandler(sqlSession, args);
      result = null;
    } else if (method.returnsMany()) {
      result = executeForMany(sqlSession, args);
    } else if (method.returnsMap()) {
      result = executeForMap(sqlSession, args);
    } else if (method.returnsCursor()) {
      result = executeForCursor(sqlSession, args);
    } else {
      Object param = method.convertArgsToSqlCommandParam(args);
      result = sqlSession.selectOne(command.getName(), param);             // selectOne
      if (method.returnsOptional() && (result == null || !method.getReturnType().equals(result.getClass()))) {
        result = Optional.ofNullable(result);
      }
    }
    break;
  case FLUSH:
    result = sqlSession.flushStatements();
    break;
  default:
    throw new BindingException("Unknown execution method for: " + command.getName());
}
if (result == null && method.getReturnType().isPrimitive() && !method.returnsVoid()) {
  throw new BindingException("Mapper method '" + command.getName()
      + "' attempted to return null from a method with a primitive return type (" + method.getReturnType() + ").");
}
return result;
  }
```

```java title="org.apache.ibatis.session.defaults.DefaultSqlSession#selectOne(java.lang.String, java.lang.Object)"
\-- org.apache.ibatis.session.defaults.DefaultSqlSession#selectList(...)

  private <E> List<E> selectList(String statement, Object parameter, RowBounds rowBounds, ResultHandler handler) {
try {
  MappedStatement ms = configuration.getMappedStatement(statement);         // 1. MappedStatement
  dirty |= ms.isDirtySelect();
  return executor.query(ms, wrapCollection(parameter), rowBounds, handler); // 2. executor query 
} catch (Exception e) {
  throw ExceptionFactory.wrapException("Error querying database.  Cause: " + e, e);
} finally {
  ErrorContext.instance().reset();
}
  }
```


## execute query


> Executor#query

```java
org.apache.ibatis.executor.Executor#query(org.apache.ibatis.mapping.MappedStatement, java.lang.Object, org.apache.ibatis.session.RowBounds, org.apache.ibatis.session.ResultHandler)
```

- 1. BaseExecutor

```java
org.apache.ibatis.executor.BaseExecutor#query(org.apache.ibatis.mapping.MappedStatement, java.lang.Object, org.apache.ibatis.session.RowBounds, org.apache.ibatis.session.ResultHandler, org.apache.ibatis.cache.CacheKey, org.apache.ibatis.mapping.BoundSql)
  list = resultHandler == null ? (List<E>) localCache.getObject(key) : null;
  if (list != null) {
    handleLocallyCachedOutputParameters(ms, key, parameter, boundSql);
  } else {
    list = queryFromDatabase(ms, parameter, rowBounds, resultHandler, key, boundSql);
  }
\-- org.apache.ibatis.executor.BaseExecutor#queryFromDatabase
\--\-- org.apache.ibatis.executor.BaseExecutor#doQuery

org.apache.ibatis.executor.SimpleExecutor#doQuery
  public <E> List<E> doQuery(MappedStatement ms, Object parameter, RowBounds rowBounds, ResultHandler resultHandler,
  BoundSql boundSql) throws SQLException {
Statement stmt = null;
try {
  Configuration configuration = ms.getConfiguration();
  StatementHandler handler = configuration.newStatementHandler(wrapper, ms, parameter, rowBounds, resultHandler,
      boundSql);                                          // RoutingStatementHandler
  stmt = prepareStatement(handler, ms.getStatementLog());
  return handler.query(stmt, resultHandler);
} finally {
  closeStatement(stmt);
}
  }

org.apache.ibatis.executor.statement.PreparedStatementHandler#query
  public <E> List<E> query(Statement statement, ResultHandler resultHandler) throws SQLException {
PreparedStatement ps = (PreparedStatement) statement;
ps.execute();                                    // java.sql.PreparedStatement do the query
return resultSetHandler.handleResultSets(ps);    // handle result
  }
```

- 2. CachingExecutor

```java
  public <E> List<E> query(MappedStatement ms, Object parameterObject, RowBounds rowBounds, ResultHandler resultHandler,
  CacheKey key, BoundSql boundSql) throws SQLException {
Cache cache = ms.getCache();
if (cache != null) {
  flushCacheIfRequired(ms);
  if (ms.isUseCache() && resultHandler == null) {
    ensureNoOutParams(ms, boundSql);
    @SuppressWarnings("unchecked")
    List<E> list = (List<E>) tcm.getObject(cache, key);
    if (list == null) {
      list = delegate.query(ms, parameterObject, rowBounds, resultHandler, key, boundSql);
      tcm.putObject(cache, key, list); // issue #578 and #116
    }
    return list;
  }
}
return delegate.query(ms, parameterObject, rowBounds, resultHandler, key, boundSql);
  }
```


# Cache


```java
// settings: localCacheScope

public enum LocalCacheScope {
  SESSION, STATEMENT
}
```

- 1: SqlSession level
- 2: MappedStatement level

Executor is create in DefaultSqlSessionFactory.

> org.apache.ibatis.executor.BaseExecutor#createCacheKey

```java title="org.apache.ibatis.executor.BaseExecutor#createCacheKey"
public CacheKey createCacheKey(MappedStatement ms, Object parameterObject, RowBounds rowBounds, BoundSql boundSql) {
    if (closed) {
      throw new ExecutorException("Executor was closed.");
    }
    CacheKey cacheKey = new CacheKey();
    cacheKey.update(ms.getId());
    cacheKey.update(rowBounds.getOffset());
    cacheKey.update(rowBounds.getLimit());
    cacheKey.update(boundSql.getSql());
    List<ParameterMapping> parameterMappings = boundSql.getParameterMappings();
    TypeHandlerRegistry typeHandlerRegistry = ms.getConfiguration().getTypeHandlerRegistry();
    // mimic DefaultParameterHandler logic
    for (ParameterMapping parameterMapping : parameterMappings) {
      if (parameterMapping.getMode() != ParameterMode.OUT) {
        Object value;
        String propertyName = parameterMapping.getProperty();
        if (boundSql.hasAdditionalParameter(propertyName)) {
          value = boundSql.getAdditionalParameter(propertyName);
        } else if (parameterObject == null) {
          value = null;
        } else if (typeHandlerRegistry.hasTypeHandler(parameterObject.getClass())) {
          value = parameterObject;
        } else {
          MetaObject metaObject = configuration.newMetaObject(parameterObject);
          value = metaObject.getValue(propertyName);
        }
        cacheKey.update(value);
      }
    }
    if (configuration.getEnvironment() != null) {
      // issue #176
      cacheKey.update(configuration.getEnvironment().getId());
    }
    return cacheKey;
  }
```

> org.apache.ibatis.executor.CachingExecutor#query()

```java
  public <E> List<E> query(MappedStatement ms, Object parameterObject, RowBounds rowBounds, ResultHandler resultHandler,
      CacheKey key, BoundSql boundSql) throws SQLException {
    Cache cache = ms.getCache();
    if (cache != null) {
      flushCacheIfRequired(ms);
      if (ms.isUseCache() && resultHandler == null) {
        ensureNoOutParams(ms, boundSql);
        @SuppressWarnings("unchecked")
        List<E> list = (List<E>) tcm.getObject(cache, key);
        if (list == null) {
          list = delegate.query(ms, parameterObject, rowBounds, resultHandler, key, boundSql);
          tcm.putObject(cache, key, list); // issue #578 and #116
        }
        return list;
      }
    }
    return delegate.query(ms, parameterObject, rowBounds, resultHandler, key, boundSql);
  }
```


# Plugin
- Configuration > plugins: https://mybatis.org/mybatis-3/configuration.html#plugins


```java
// 1. the interceptor chain
org.apache.ibatis.session.Configuration
  protected final InterceptorChain interceptorChain = new InterceptorChain();

// 2. where to add interceptor
// call org.apache.ibatis.session.Configuration#addInterceptor
org.apache.ibatis.builder.xml.XMLConfigBuilder#pluginElement - <plugin interceptor="xxx">
  private void pluginElement(XNode parent) throws Exception {
if (parent != null) {
  for (XNode child : parent.getChildren()) {
    String interceptor = child.getStringAttribute("interceptor");
    Properties properties = child.getChildrenAsProperties();
    Interceptor interceptorInstance = (Interceptor) resolveClass(interceptor).getDeclaredConstructor()
        .newInstance();
    interceptorInstance.setProperties(properties);
    configuration.addInterceptor(interceptorInstance);
  }
}
  }

// 3. where to add interceptors
// call InterceptorChain#pluginAll
org.apache.ibatis.session.Configuration#newParameterHandler
org.apache.ibatis.session.Configuration#newResultSetHandler
org.apache.ibatis.session.Configuration#newStatementHandler
org.apache.ibatis.session.Configuration#newExecutor(org.apache.ibatis.transaction.Transaction, org.apache.ibatis.session.ExecutorType)
```

```java
org.apache.ibatis.session.Configuration#newExecutor
\-- (Executor) interceptorChain.pluginAll(executor)
```
